In [1052]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 12
year = 2022

data = get_data(day=day, year=year)

# submit(cube.sum(), part="a", day=day, year=year)

# print(data)

In [1031]:
chr(53
    )

'5'

In [1053]:
# grid = np.array([[c if c in "ES" else ord(c) - ord('a') for c in l] for l in data.split('\n')])



abaaaaac
abaaaaaa
abaaaaaa
abaaacac
abaaaccc


In [1070]:
grid = {(i,j):ord(c) - ord('a') for j,l in enumerate(data.split('\n')) for i,c in enumerate(l)}
end, start = [next(k for k,v in grid.items() if v == i) for i in (-28,-14)]
grid[end] = 25
grid[start] = 0

add = lambda x,d: (x[0]+d[0], x[1]+d[1])

dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]
stack = [end]
dists = defaultdict(lambda: sys.maxsize)
dists[end] = 0

while len(stack):
    coords = stack.pop()
    for dir in dirs:
        neigh = add(coords, dir)
        if neigh not in grid:
            continue
        if (dists[neigh] > dists[coords] + 1) and (grid[neigh] >= grid[coords] - 1):
            dists[neigh] = dists[coords] + 1
            stack.append(neigh)
    
print('Part 1:', dists[start])
print('Part 2:', min(dists[k] for k,v in grid.items() if v == 0))

Part 1: 472
Part 2: 465


In [1067]:
min([dists[k] for k,v in grid.items() if v == 0])

465

In [1060]:
len(grid)

6314

['D', 'C']